In [ ]:
import numpy as np
import subprocess
import time
import copy
from utils.opt_utils import *
import os
import h5py
import pickle
from qutip import *

wc_A = 4.069814 * (10 ** 9) * 2 * np.pi  # cavity A frequency
wc_B = 6.096062 * (10 ** 9) * 2 * np.pi  # cavity A frequency
wa = 5.325 * (10 ** 9) * 2 * np.pi  # atom frequency
dt_A = np.abs(wc_A - wa) / (2 * np.pi)
dt_B = np.abs(wc_B - wa) / (2 * np.pi)
chi_A = 0.00215 * (10 ** 9) * 2 * np.pi
chi_B = 0.00544 * (10 ** 9) * 2 * np.pi
g_A = np.sqrt(chi_A * dt_A) * 2 * np.pi  # coupling strength w/ cavity A
g_B = np.sqrt(chi_B * dt_B) * 2 * np.pi  # coupling strength w/ cavity B

gamma = 333333.333  # atom dissipation rate
kappa_A = 10000  # cavity A dissipation rate
kappa_B = 10000  # cavity B dissipation rate

temp_q = 0.01  # avg number of thermal bath excitation for qubit
temp_A = 0.04  # avg number of thermal bath excitation for cavity A
temp_B = 0.05  # avg number of thermal bath excitation for cavity B
cavity_dims = 8


# Cost function
def cost_q_e(final_expect, final_dm):
    # print(final_expect[0])
    return (final_expect[0])


def cost_qA_g1(final_expect, final_dm):
    # print(final_dm.full())
    return np.abs(final_dm.full()[1][0])


def cost_qAB_g11(final_expect, final_dm):
    return np.power(np.abs(final_dm.full()[cavity_dims + 1][0]), 2)


def cost_qAB_g11_dm(final_expect, final_state):
    return np.power(np.abs(final_state[cavity_dims + 1][0]), 2)


def cost_qAB_g11_n(final_expect, final_dm):
    noise = (np.random.rand(1)[0] * 0.10) - 0.05
    return np.abs(final_dm.full()[cavity_dims + 1][0]) + noise


# ========== OPTIONS ========== #
max_segs = 20
time_start = 0.0000000
time_stop = 0.000001
init_amp = 4000000
n_steps = 501

num_drives = 3
num_elems = 3
num_cavities = 2
# cavity_dims = 8
state_sizes = [2, cavity_dims, cavity_dims]
state_vals = [0, 0, 0]
init_freqs = [wa, wc_A, wc_B]
sim_options = Options()
element_freqs = [wa, wc_A, wc_B]
output_cost_func = cost_qAB_g11_dm
elements = "qAB"
start_split_num = 10
n_seg_jump = 1
verbose = True

save_dir = r'C:\Users\Wang_Lab\Documents\GitLab\quantum_control_rl_server\examples\ctrl_vqe_pi_pulse_sim_interp\save_data'
hdf5_name = time.strftime('g11-n0-%Y%m%d-%H%M%S.h5')
epochs = 1000
epochs_per_seg = 500
train_batch_size = 20
qubit_amp_scale = 4
cavity_amp_scale = 4
freq_scale = 0.0005
# ========== OPTIONS ========== #

t_arr = np.linspace(time_start, time_stop, n_steps)

t_step = (time_stop - time_start) / n_steps

sim_options.store_final_state = True

qscale = []
cscale = []
for i in range(2 * start_split_num):
    qscale.append(init_amp * qubit_amp_scale)
    cscale.append(init_amp * cavity_amp_scale)
    cscale.append(init_amp * cavity_amp_scale)

sm, a_A, a_B, sx, sz = reg_ops(num_cavities + 1, cavity_dims)
drive_freqs = np.array(init_freqs)

gammas = [gamma, kappa_A, kappa_B]
temps = [temp_q, temp_A, temp_B]
c_ops = []  # gen_c_ops(elements, [sm, a_A, a_B, sx, sz], gammas, temps)

# Operators used in Hamiltonian
drive_ops = [sm.dag(), sm, a_A.dag(), a_A, a_B.dag(), a_B]
element_ops = [sz, a_A.dag() * a_A, a_B.dag() * a_B]
H_0 = (chi_A * a_A.dag() * a_A * sz / 2) + (chi_B * a_B.dag() * a_B * sz / 2)
eval_ops = [sm.dag() * sm, a_A.dag() * a_A,
            a_B.dag() * a_B]  # [sm.dag() * sm, a_A.dag() * a_A, a_B.dag() * a_B] # [sm.dag() * sm, a_A.dag() * a_A, a_B.dag() * a_B, tensor(destroy(2) * destroy(2).dag(), destroy(cavity_dims).dag() * destroy(cavity_dims), destroy(cavity_dims).dag() * destroy(cavity_dims))]

t_segs, amp_segs = setup_interp_segs(2 * num_drives, time_start, time_stop, init_amp)

t_segs = t_segs[:, 1:-1]

# for i in range(start_split_num - 1):
#     t_segs, amp_segs = split_segs(t_segs, amp_segs)

# Setup initial state
# init_state = build_psi(state_sizes, state_vals)
init_state = tensor(
    (basis(state_sizes[0], 0) * np.sqrt(1 - state_vals[0])) + (basis(state_sizes[0], 1) * np.sqrt(state_vals[0])),
    (basis(state_sizes[1], 0) * np.sqrt(1 - state_vals[1])) + (basis(state_sizes[1], 1) * np.sqrt(state_vals[1])),
    (basis(state_sizes[2], 0) * np.sqrt(1 - state_vals[2])) + (basis(state_sizes[2], 1) * np.sqrt(state_vals[2])))

amp_segs = np.reshape(amp_segs, (num_drives * 2, int(len(amp_segs.flatten()) / (num_drives * 2))))

t_segs = np.reshape(t_segs, (num_drives * 2, int(len(t_segs.flatten()) / (num_drives * 2))))

for i in range(start_split_num - 1):
    for i in range(n_seg_jump):
        
        t_segs, amp_segs = split_segs_flat(interp_time_wrapper(t_segs, time_start, time_stop),
                                           interp_amp_wrapper(amp_segs))
        t_segs = t_segs[:, 1:-1]
t_segs = interp_time_wrapper(t_segs, time_start, time_stop)

flip_bits = np.array([[False, False, True, True]])
flip_mask = np.array([np.repeat(flip_bits, np.ceil(start_split_num / 4), axis=0).flatten()[:start_split_num]])
flip_mask = np.repeat([flip_mask], num_drives * 2, axis=1)[0]

amp_segs[flip_mask] *= -1

# Create blank history arrays for storing optimal / past values
time_hist = []
amp_hist = []
freq_hist = []
cost_hist = []